# **Import station data from Data.gov.sg**
This script extract station data from Air Temerature across Singapore real-time API and ingest into PostgreSQL data_gov_project database as a dimension table.

In [ ]:
import requests  # For sending HTTP requests
import pandas as pd  # For data manipulation
from sqlalchemy import create_engine  # For database operations

# Define constants
API_URL = "https://api-open.data.gov.sg/v2/real-time/api/air-temperature"
DB_USER = 'postgres'  # Update with your PostgreSQL username
DB_PASS = 'admin'      # Update with your PostgreSQL password
DB_HOST = 'localhost'  # Update with your database host
DB_PORT = '5432'       # Update with your database port
DB_NAME = 'data_gov_project'  # Update with your PostgreSQL database name

def get_station_metadata(json_data):
    """Fetch station metadata and return a DataFrame."""
    stations = json_data.get('data', {}).get('stations', [])
    station_data = []

    for station in stations:
        station_data.append({
            'station_id': station.get('id'),  # Use 'id'
            'device_id': station.get('deviceId'),
            'station_name': station.get('name'),
            'latitude': station['location']['latitude'],  # Use 'location' key for latitude
            'longitude': station['location']['longitude']   # Use 'location' key for longitude
        })

    return pd.DataFrame(station_data)

def fetch_station_data(api_url):
    """Fetch station data from the API."""
    response = requests.get(api_url)
    if response.status_code == 200:
        json_data = response.json()
        print(json_data)  # Print the full JSON response for inspection
        station_df = get_station_metadata(json_data)  # Get the station metadata

        return station_df
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame on failure

def load_data_to_postgres(data_frame):
    """Load the provided pandas DataFrame into the 'station' table in your PostgreSQL database."""
    engine = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}')
    try:
        data_frame.to_sql('station', engine, if_exists='append', index=False)  # Append mode
        print(f"Successfully loaded {len(data_frame)} records to PostgreSQL table.")
    except Exception as e:
        print(f"Error loading data into PostgreSQL: {e}")

def main():
    print("Starting the script...")

    # Fetch and process the data
    station_df = fetch_station_data(API_URL)

    if not station_df.empty:
        load_data_to_postgres(station_df)  # Load the data into PostgreSQL
    else:
        print("No data collected.")
    
    print("Script completed.")

if __name__ == "__main__":
    main()